PyEOM is Python package for post-Hartree-Fock calculations of excited states based on the equation-of-motion framework.

This tutorial is a quick introduction to PyEOM library, and introduces the following topics:
* Specifiying a molecular system (Hamiltonian and RDMs)
* Evaluating ionization potentials
* Evaluating excitation energies

<!-- * Calculating the transition density matrix for the oscillator strength -->


## Specifiying a molecular system

All high-level methods in this package require:
* the electronic Hamiltonian
* the approximated wavefunction for the reference (ground) state

The Hamiltonian must be specified through the one- and two-electron integrals and the reference ground state through its 1- and 2-electron reduced density matrices (RDMs).


In the next cell, the Hamiltonian for the Beryllium atom is loaded from Numpy arrays. Note that using [IOData](https://iodata.readthedocs.io/en/latest/) the Hamiltonian input parameters can alternatively be loaded from a FCIDUMP file generated by an external quantum chemistry package.

In [ ]:
# Import statements
import numpy as np
from eomee.tools import spinize
from eomee.tools import find_datafiles


# Load the one- and two-electron integrals in the restricted MO basis
one_int = np.load(find_datafiles('be_sto3g_oneint.npy'))
two_int = np.load(find_datafiles('be_sto3g_twoint.npy'))
one_sp = spinize(one_int) # (k,k) --> (2k, 2k) matrix dimensions
two_sp = spinize(two_int) # (k,k,k,k) --> (2k, 2k, 2k, 2k) array dimensions

The loaded electron integrals (`one_int` and `two_int`) are given in a restricted molecular orbital basis (as is commonly the case with post-HF approaches) while PyEOM methods are represented in terms of generalized spin-orbital. The utility function `spinize` can be used to facilitate the mapping to our internal format.

In this package, the $N$-electron ground state is defined through its one- and two-electron reduced density matrices (RDMs).
The RDMs have to be loaded from Numpy arrays. However, for single Slater determinant wavefunction models like the Hartree-Fock wavefunction, PyEOM provides the tool function `hartreefock_rdms` to generate the RDMs.

In [ ]:
from eomee.tools import hartreefock_rdms

# Generate the Hartree-Fock 1- and 2-RDMs
nelecs = (2, 2)
nbasis = one_int.shape[0]
rdm1, rdm2 = hartreefock_rdms(nbasis, *nelecs)

## Computing spectroscopic properties

### Ionization Potentials

To compute the ionization potentials (IP) use one of the method `IP`, `IPc` or `IPa`. An istances of these classes encodes the generalized eigenvalue problem that must be solved to get the transition energies and the associated transition operators corresponding to removing one electron from the ground state.

The next cell illustrates how to create an instance of the class `IP` and how to access some of its attributes such as the matrices for the eigenvalue problem representing this method. (For the HF reference `IP` is equivalent to the Koomans' Theorem).

In [ ]:
from eomee.ionization import IP

ekt = IP(one_sp, two_sp, rdm1, rdm2)

print("Generalized Fock matrix:\n", ekt.lhs)
print("Metric matrix:\n", ekt.rhs)
print("Number of eigenvalue/vectors", ekt.neigs)

All ionization energies can be computed at once with the `solve_dense` method.

In [ ]:
# Solve the EKT eigenvalue problem
ev, cv = ekt.solve_dense()

# Print the first 5 ionization energies
print("{0:<2s} {1:<6s}".format('n', 'EDiff.(a.u.)'))
for i, e in enumerate(ev[:5]):
    print("{0:<2} {1:<6.3}".format(i, e))

### Electronic excitation energies

To get excitation energies use the method `EE` (or the alternative `EEm` method). The istance of this class encodes the generalized eigenvalue problem that must be solved to get the excitation energies and the corresponding transition coefficients.

In [ ]:
from eomee.excitation import EE

erpa = EE(one_sp, two_sp, rdm1, rdm2)

print("Diagonal elements of orbital Hessian matrix:\n", np.diag(erpa.lhs))
print("Diagonal elements of metric matrix:\n", np.diag(erpa.rhs))
print("Number of eigenvalue/vectors", erpa.neigs)

All excitation energies can be computed at once with the `solve_dense` method or only a few of them with `solve_sparse`. The number of lowest excitation energies to be computed is specified by the parameter `nsols`.

In [ ]:
(e1, e2) = ekt.solve_sparse(nsols=2)[0]

print("Two lowest excitations: ", e1, e2)

For closed-shell systems like Beryllium, one can use the spin-adapted implementations `EES` and `EET` to compute the singlet and triplet excitation energies, respectively. These EOM methods are subclasses of `EE`-EOM and therefore inherit its methods.

In [ ]:
from eomee.spinadapted.particlehole import EET

myeom = EET(one_int, two_sp, rdm1, rdm2)
ev1, cv1 = myeom.solve_dense()

# Print the first 5 triplet excitation energies
print("{0:<2s} {1:<6s}".format('n', 'EDiff.(a.u.)'))
for i, e in enumerate(ev1[:5]):
    print("{0:<2} {1:<6.3}".format(i, e))

## Post-processing EOM results

For a transition process of interest, it is also posible to compute the transition density matrices (TDMs) with the `compute_tdm` method.

The next cell illustrates how to compute the TDM between the ground state and the lowest $(N-1)$-electron 
excited state from the eigenvectors of the `IP`-EOM equation.

In [ ]:
tdm = ekt.compute_tdm1(cv1[0])